In [1]:
#!/usr/bin/env python
# coding: utf-8

# 
# ## Attention!
# ### This code runs in MPI mode.
# 
# 
# Trying to recover the input values of the simulation. The free parameters are:
#    - One ML, One beta, qinc, mbh, kappa_s, qDm, mag_shear, phi_shear and gamma.
# 

# In[1]:


#Control time packages
import time
import os
os.environ["OMP_NUM_THREADS"] = "1"

#MPI
from schwimmbad import MPIPool

#General packages
import numpy as np
import emcee
import matplotlib.pyplot as plt

#Constants and usefull packages
from astropy.cosmology import Planck15 as cosmo
from astropy.cosmology import z_at_value
from astropy.constants import G, M_sun, c
import astropy.units as u

#Autolens Model packages
import autolens as al
import autolens.plot as aplt

#My Emcee for Pyautolens
import My_Autolens

data_folder = "/home/carlos/Documents/GitHub/Master-Degree/Autolens tests/autolens_workspace/Test_5/Simulation_Data/"

In [2]:
    #Reading MGE inputs
surf_lum, sigma_lum, qobs_lum = np.loadtxt("Input/JAM_Input.txt", unpack=True)      #MGE decomposition
surf_dm, sigma_dm , qobs_dm   = np.loadtxt("Input/eNFW.txt", unpack=True)            #DM component

## Models inicialization

"""
    To inicialize the model, we set some random values for the parameters. But it's only necessary for initialize the model. During the non-linear search, this values will be updated constantly until the best fit.
"""  
#Only for lensing modelling 
z_l    = 0.299                                                         #Lens Redshift
z_s    = 4.100                                                         #Source Redshift 
D_l    = cosmo.angular_diameter_distance(z_l).value                    #Distance to lens [Mpc] 
mbh    = 1e9                                                           #mass of black hole [log10(M_sun)]
kappa_ = 0.075                                                         #kappa_s of DM profile
ml     = 7.00                                                          #mass to light ratio
r_s    = 11.5                                                          #scale radius [arcsec]
shear_comp = al.convert.shear_elliptical_comps_from(magnitude=0.02, phi=88) #external shear


#Autolens Data
imaging = al.Imaging.from_fits(
        image_path=f"{data_folder}/arcs_simulation.fits",
        noise_map_path=f"{data_folder}/noise_simulation.fits",
        psf_path=f"{data_folder}/psf_simulation.fits",
        pixel_scales=0.1,
    )

mask        = al.Mask.from_fits( file_path=f"{data_folder}/new_mask.fits", hdu=1, 
                                pixel_scales=imaging.pixel_scales)

masked_image = al.MaskedImaging(imaging=imaging, mask=mask, inversion_uses_border=True)   #Masked image

#--------------------------------------------------------------------------------------------------#
# PYAUTOLENS MODEL
#MGE mass profile
mass_profile = al.mp.MGE()    #Mass class

ell_comps    = al.convert.elliptical_comps_from(axis_ratio=qobs_dm[0], phi=0.0) #Elliptical components in Pyautolens units
eNFW         = al.mp.dark_mass_profiles.EllipticalNFW(kappa_s=kappa_, elliptical_comps=ell_comps ,scale_radius=r_s) #Analytical eNFW profile


#Components
#Do not include MGE DM component here
mass_profile.MGE_comps(z_l=z_l, z_s=z_s, 
                       surf_lum=surf_lum, sigma_lum=sigma_lum, qobs_lum=qobs_lum, ml=ml, mbh=mbh) 
mass_profile.Analytic_Model(eNFW)  #Include Analytical NFW

#--------------------------------------------------------------------------------------------------#
#Emcee Model
emcee_model = My_Autolens.Models(mass_profile=mass_profile, masked_imaging=masked_image, quiet=True)
emcee_model.include_DM_analytical(eNFW)

In [3]:
filename = "Input/Lens_Simulation.h5"
read = emcee.backends.HDFBackend(filename)
backend = emcee.backends.HDFBackend(filename)
nwalkers, ndim = read.shape

In [4]:
moves=[(emcee.moves.DEMove(gamma0=0.063), 0.90), (emcee.moves.DEMove(), 0.10)]
    #Initialize the new sampler
new_sampler = emcee.EnsembleSampler(nwalkers, ndim, emcee_model,backend=backend, moves=moves)

In [5]:
state = new_sampler.get_last_sample()

In [6]:
new_sampler.reset()
backend.reset(nwalkers=nwalkers, ndim=ndim)

In [ ]:
nsteps = 500000                          #Number of walkes 

 # We'll track how the average autocorrelation time estimate changes
index = 0
autocorr = np.empty(nsteps)
 # This will be useful to testing convergence
old_tau = np.inf
 # This saves how many walkers have been accepted in the last 100 steps
old_accp = np.zeros(nwalkers,)


# Now we'll sample for up to max_n steps
start = time.time()
global_time = time.time()


for sample in new_sampler.sample(state, iterations=nsteps, progress=True):
    # Only check convergence every 100 steps
    if new_sampler.iteration % 10:
        continue
    print("\n")
    print("##########################")

    #Compute how many walkes have been accepted during the last 100 steps

    new_accp = new_sampler.backend.accepted             #Total number of accepted
    old_accp = new_accp - old_accp                  #Number of accepted in the last 100 steps
    mean_accp_100 = np.mean(old_accp/float(100))    #Mean accp fraction of last 100 steps

    #Update a table output with acceptance
    table = np.loadtxt("Output_LogFile.txt")

    iteration = new_sampler.iteration
    accept = np.mean(new_sampler.acceptance_fraction)
    total_time = time.time() - global_time
    upt = np.column_stack([iteration, accept, total_time, mean_accp_100])

    np.savetxt('Output_LogFile.txt', np.vstack([table, upt]),
                            fmt=b'	%i	 %e			 %e             %e', 
                        header="Output table for the combined model: Dynamic.\n Iteration	 Mean acceptance fraction	 Processing Time    Last 100 Mean Accp. Fraction")

    # Compute the autocorrelation time so far
    # Using tol=0 means that we'll always get an estimate even
    # if it isn't trustworthy
    tau = new_sampler.get_autocorr_time(tol=0)
    autocorr[index] = np.mean(tau)
    index += 1

    # Check convergence
    converged = np.all(tau * 100 < new_sampler.iteration)
    converged &= np.all(np.abs(old_tau - tau) / tau < 0.01)
    if converged:
        if 0.2 < accept < 0.35:
            break
    old_tau = tau



end = time.time()
print('\n')
print("Final")
multi_time = end - start
print("Multiprocessing took {0:.1f} seconds".format(multi_time))


  0%|          | 10/500000 [10:10<8589:44:28, 61.85s/it]/home/carlos/anaconda3/lib/python3.7/site-packages/emcee/autocorr.py:36: RuntimeWarning: invalid value encountered in true_divide
  acf /= acf[0]
/home/carlos/anaconda3/lib/python3.7/site-packages/emcee/autocorr.py:41: RuntimeWarning: invalid value encountered in less
  m = np.arange(len(taus)) < c * taus
/home/carlos/anaconda3/lib/python3.7/site-packages/emcee/autocorr.py:99: RuntimeWarning: invalid value encountered in greater
  flag = tol * tau_est > n_t
/home/carlos/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in less
/home/carlos/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:51: RuntimeWarning: invalid value encountered in less




##########################


  0%|          | 20/500000 [19:19<8202:05:27, 59.06s/it]



##########################


  0%|          | 30/500000 [29:47<8759:13:29, 63.07s/it]



##########################


  0%|          | 40/500000 [40:11<8720:39:09, 62.79s/it]



##########################


  0%|          | 46/500000 [46:35<8909:17:59, 64.15s/it]

An exception ocurres in Pyautolens_log_likelihood().
An exception ocurres in Pyautolens_log_likelihood().
An exception ocurres in Pyautolens_log_likelihood().
